In [1]:
import random
random.seed(0)
import numpy as np
np.random.seed(1)
import tensorflow as tf
tf.set_random_seed(2)

import sys
sys.path.append('..')
import config
from fqc import uccsd, qaoa, util

import numpy as np
from datetime import datetime

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [2]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [3]:
def binary_search_for_shortest_pulse_time(min_time, max_time, convergence, reg_coeffs):
    """Search between [min_time, max_time] up to 1ns tolerance. Assumes 20 steps per ns.""" 
    min_steps, max_steps = min_time * 20, max_time * 20
    while min_steps + 20 < max_steps: # just estimate to +- 1ns
        mid_steps = int((min_steps + max_steps) / 2)
        total_time = mid_steps / 20.0
        print('\n\ntrying total_time: %s for unitary of size %s' % (str(total_time), str(U.shape)))
        SS = Grape(H0, Hops, Hnames, U, total_time, mid_steps, states_concerned_list, convergence,
                    reg_coeffs=reg_coeffs,
                    use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA, show_plots=False, file_name=file_name, data_path=data_path)
        if SS.l < SS.conv.conv_target: # if converged, search lower half 
            max_steps = mid_steps
        else:
            min_steps = mid_steps
    return mid_steps / 20

## 3Reg Full

In [ ]:
full_times = []
for N in range(4, 5, 2):
    row = 2
    col = int(N / 2)
    d = 2 # this is the number of energy levels to consider (i.e. d-level qudits) 
    max_iterations = (2**N) * 125
    decay = max_iterations / 2
    convergence = {'rate':0.01, 'max_iterations': max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay, 'min_grad': 1e-9, 'update_step': 20}
    reg_coeffs = {}
    for p in range(1,7):
        circuit = qaoa.get_qaoa_circuit(N, p, '3Reg')
        # TODO: average over 10 graphs
        U = util.circuitutil.get_unitary(circuit)
        connected_qubit_pairs = util.get_nearest_neighbor_coupling_list(row, col, directed=False)
        H0 = hamiltonian.get_H0(N, d)
        Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
        states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
        maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)
        shortest_time = binary_search_for_shortest_pulse_time(10, int(90*(N/5)*p), convergence, reg_coeffs)
        full_times.append(shortest_time)



trying total_time: 41.0 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/yongshan/00006_Mar13.h5
Using 5 Taylor terms and 4 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.78e-01; Runtime: 2.5s; Iterations = 0, grads =   2.068e-02, unitary_metric = 1.00049
Error = :5.78e-01; Runtime: 35.2s; Iterations = 20, grads =   9.584e-01, unitary_metric = 1.00028
Error = :3.99e-01; Runtime: 63.1s; Iterations = 40, grads =   2.401e-01, unitary_metric = 1.00044
Error = :3.76e-01; Runtime: 91.1s; Iterations = 60, grads =   2.634e-02, unitary_metric = 1.00045
Error = :3.74e-01; Runtime: 119.0s; Iterations = 80, grads =   4.331e-03, unitary_metric = 1.00041
Error = :3.80e-01; Runtime: 147.1s; Iterations = 100, grads =   2.56

Error = :3.74e-01; Runtime: 162.5s; Iterations = 140, grads =   2.493e-02, unitary_metric = 1.00018
Error = :3.74e-01; Runtime: 184.8s; Iterations = 160, grads =   3.606e-05, unitary_metric = 1.00018
Error = :3.74e-01; Runtime: 207.0s; Iterations = 180, grads =   3.280e-03, unitary_metric = 1.00018
Error = :3.74e-01; Runtime: 229.3s; Iterations = 200, grads =   2.625e-02, unitary_metric = 1.00018
Error = :3.74e-01; Runtime: 253.1s; Iterations = 220, grads =   1.635e-03, unitary_metric = 1.00019
Error = :3.74e-01; Runtime: 275.4s; Iterations = 240, grads =   2.105e-04, unitary_metric = 1.00020
Error = :3.76e-01; Runtime: 297.6s; Iterations = 260, grads =   6.421e-02, unitary_metric = 1.00019
Error = :3.74e-01; Runtime: 319.9s; Iterations = 280, grads =   9.416e-04, unitary_metric = 1.00019
Error = :3.74e-01; Runtime: 342.2s; Iterations = 300, grads =   3.411e-04, unitary_metric = 1.00019
Error = :3.76e-01; Runtime: 366.0s; Iterations = 320, grads =   6.080e-02, unitary_metric = 1.00019


Error = :3.74e-01; Runtime: 713.3s; Iterations = 760, grads =   7.076e-07, unitary_metric = 1.00002
Error = :3.74e-01; Runtime: 731.8s; Iterations = 780, grads =   1.290e-08, unitary_metric = 1.00002
Error = :3.74e-01; Runtime: 750.2s; Iterations = 800, grads =   8.100e-09, unitary_metric = 1.00002
Error = :3.74e-01; Runtime: 764.4s; Iterations = 814, grads =   8.923e-10, unitary_metric = 1.00002
data saved at: /project/ftchong/qoc/yongshan/00010_Mar13.h5


trying total_time: 28.35 for unitary of size (16, 16)
data saved at: /project/ftchong/qoc/yongshan/00011_Mar13.h5
Using 7 Taylor terms and 2 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.86e-01; Runtime: 1.7s; Iterations = 0, grads =   1.255e-02, unitary_metric = 1.00003


Error = :1.44e-03; Runtime: 3070.0s; Iterations = 1080, grads =   9.211e-07, unitary_metric = 1.00042
Error = :1.40e-03; Runtime: 3125.7s; Iterations = 1100, grads =   8.380e-07, unitary_metric = 1.00039
Error = :1.36e-03; Runtime: 3185.4s; Iterations = 1120, grads =   7.695e-07, unitary_metric = 1.00039
Error = :1.30e-03; Runtime: 3241.1s; Iterations = 1140, grads =   7.143e-07, unitary_metric = 1.00042
Error = :1.26e-03; Runtime: 3297.0s; Iterations = 1160, grads =   6.636e-07, unitary_metric = 1.00041
Error = :1.22e-03; Runtime: 3352.7s; Iterations = 1180, grads =   6.263e-07, unitary_metric = 1.00042
Error = :1.19e-03; Runtime: 3408.7s; Iterations = 1200, grads =   5.946e-07, unitary_metric = 1.00042
Error = :1.15e-03; Runtime: 3468.0s; Iterations = 1220, grads =   5.614e-07, unitary_metric = 1.00043
Error = :1.12e-03; Runtime: 3524.0s; Iterations = 1240, grads =   5.388e-07, unitary_metric = 1.00042
Error = :1.09e-03; Runtime: 3579.7s; Iterations = 1260, grads =   5.158e-07, unita

Error = :1.52e-02; Runtime: 1855.9s; Iterations = 1240, grads =   3.323e-06, unitary_metric = 1.00022
Error = :1.50e-02; Runtime: 1885.4s; Iterations = 1260, grads =   3.050e-06, unitary_metric = 1.00023
Error = :1.48e-02; Runtime: 1914.9s; Iterations = 1280, grads =   3.365e-06, unitary_metric = 1.00023
Error = :1.45e-02; Runtime: 1944.2s; Iterations = 1300, grads =   4.348e-06, unitary_metric = 1.00023
Error = :1.43e-02; Runtime: 1975.5s; Iterations = 1320, grads =   6.070e-06, unitary_metric = 1.00022
Error = :1.40e-02; Runtime: 2004.8s; Iterations = 1340, grads =   8.490e-06, unitary_metric = 1.00020
Error = :1.35e-02; Runtime: 2034.2s; Iterations = 1360, grads =   1.117e-05, unitary_metric = 1.00019
Error = :1.29e-02; Runtime: 2063.5s; Iterations = 1380, grads =   1.342e-05, unitary_metric = 1.00019
Error = :1.22e-02; Runtime: 2092.8s; Iterations = 1400, grads =   1.415e-05, unitary_metric = 1.00020
Error = :1.15e-02; Runtime: 2124.1s; Iterations = 1420, grads =   1.231e-05, unita

Error = :4.79e-03; Runtime: 1726.4s; Iterations = 760, grads =   3.422e-06, unitary_metric = 1.00027
Error = :4.60e-03; Runtime: 1773.5s; Iterations = 780, grads =   3.458e-06, unitary_metric = 1.00027
Error = :4.41e-03; Runtime: 1820.5s; Iterations = 800, grads =   3.468e-06, unitary_metric = 1.00026
Error = :4.23e-03; Runtime: 1871.4s; Iterations = 820, grads =   3.419e-06, unitary_metric = 1.00025
Error = :4.05e-03; Runtime: 1918.9s; Iterations = 840, grads =   3.342e-06, unitary_metric = 1.00024
Error = :3.86e-03; Runtime: 1966.5s; Iterations = 860, grads =   3.212e-06, unitary_metric = 1.00023
Error = :3.66e-03; Runtime: 2014.1s; Iterations = 880, grads =   3.028e-06, unitary_metric = 1.00024
Error = :3.48e-03; Runtime: 2061.6s; Iterations = 900, grads =   2.830e-06, unitary_metric = 1.00024
Error = :3.33e-03; Runtime: 2112.5s; Iterations = 920, grads =   2.586e-06, unitary_metric = 1.00023
Error = :3.16e-03; Runtime: 2160.2s; Iterations = 940, grads =   2.345e-06, unitary_metric 

Error = :3.92e-02; Runtime: 983.7s; Iterations = 480, grads =   3.067e-04, unitary_metric = 1.00052
Error = :3.92e-02; Runtime: 1023.4s; Iterations = 500, grads =   1.315e-04, unitary_metric = 1.00052
Error = :3.92e-02; Runtime: 1066.6s; Iterations = 520, grads =   1.633e-05, unitary_metric = 1.00053
Error = :3.92e-02; Runtime: 1106.7s; Iterations = 540, grads =   9.361e-06, unitary_metric = 1.00053
Error = :3.91e-02; Runtime: 1146.7s; Iterations = 560, grads =   2.780e-05, unitary_metric = 1.00053
Error = :3.91e-02; Runtime: 1186.8s; Iterations = 580, grads =   1.136e-03, unitary_metric = 1.00053
Error = :4.04e-02; Runtime: 1226.8s; Iterations = 600, grads =   1.111e-01, unitary_metric = 1.00052
Error = :3.94e-02; Runtime: 1270.0s; Iterations = 620, grads =   2.156e-02, unitary_metric = 1.00053
Error = :3.91e-02; Runtime: 1310.0s; Iterations = 640, grads =   1.869e-03, unitary_metric = 1.00053
Error = :3.91e-02; Runtime: 1349.9s; Iterations = 660, grads =   1.590e-04, unitary_metric =

Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.95e-01; Runtime: 4.5s; Iterations = 0, grads =   1.199e-02, unitary_metric = 1.00060
Error = :2.12e-01; Runtime: 54.6s; Iterations = 20, grads =   8.950e+00, unitary_metric = 1.00041
Error = :5.68e-02; Runtime: 97.6s; Iterations = 40, grads =   1.047e+00, unitary_metric = 1.00042
Error = :4.37e-02; Runtime: 140.6s; Iterations = 60, grads =   1.990e-01, unitary_metric = 1.00045
Error = :4.10e-02; Runtime: 183.6s; Iterations = 80, grads =   3.240e-02, unitary_metric = 1.00043
Error = :4.05e-02; Runtime: 226.6s; Iterations = 100, grads =   2.873e-03, unitary_metric = 1.00043
Error = :4.05e-02; Runtime: 272.6s; Iterations = 120, grads =   4.745e-04, unitary_metric = 1.00043
Error = :4.04e-02; Runtime: 315.5s; Iterations = 140, grads =   5.996e-05, unitary_metric = 1.00043
Error = :4.04e-02; Runtime: 358.4s; Iterations = 160, grads =   1.531e-05, unitary_metric =

Error = :3.97e-02; Runtime: 3502.9s; Iterations = 1600, grads =   1.873e-08, unitary_metric = 1.00044
Error = :3.97e-02; Runtime: 3548.7s; Iterations = 1620, grads =   1.874e-08, unitary_metric = 1.00045
Error = :3.97e-02; Runtime: 3591.7s; Iterations = 1640, grads =   1.920e-08, unitary_metric = 1.00044
Error = :3.97e-02; Runtime: 3634.5s; Iterations = 1660, grads =   1.956e-08, unitary_metric = 1.00044
Error = :3.97e-02; Runtime: 3677.3s; Iterations = 1680, grads =   1.910e-08, unitary_metric = 1.00044
Error = :3.97e-02; Runtime: 3720.3s; Iterations = 1700, grads =   1.989e-08, unitary_metric = 1.00045
Error = :3.97e-02; Runtime: 3766.2s; Iterations = 1720, grads =   1.995e-08, unitary_metric = 1.00044
Error = :3.96e-02; Runtime: 3808.9s; Iterations = 1740, grads =   2.024e-08, unitary_metric = 1.00045
Error = :3.96e-02; Runtime: 3851.6s; Iterations = 1760, grads =   1.988e-08, unitary_metric = 1.00045
Error = :3.96e-02; Runtime: 3894.4s; Iterations = 1780, grads =   2.040e-08, unita

In [ ]:
print(pulse_times)

### N=6

In [4]:
import os, re
times = [[] for _ in range(8)]

for filename in os.listdir('QAOA/3Reg/'):
    
    # job 59098496 was run for 3 regular graph with N=6
    if '59098496' in filename and filename.endswith('.out'):
        #print(filename)
        lines = open('QAOA/3Reg/%s' % filename).readlines()
        if len(lines) > 2 and lines[-2].startswith('TIME FOR SLICE IS:'):
            #print(re.split(';|=| |\*|\n', lines[1]))
            pi = int(re.split(';|=| |\*|\n', lines[1])[-4])
            #print(pi)
            time = float(lines[-1])
            #print(time)
            if time <= 100.0:
                times[pi-1].append(time)
n_s = [7,10,13,16,19,22,25,28]
all_times = []
for (i, t) in enumerate(times):
    print('----------')
    print("3Reg (N=6, p=%d): %s" % (i+1, np.round(sum(times[i]),4)))
    print("%d out of %d" % (len(times[i]), n_s[i]))
    print(times[i])
    all_times.append(np.round(sum(times[i]),4))

----------
3Reg (N=6, p=1): 72.0
7 out of 7
[11.05, 4.7, 7.25, 12.35, 4.4, 3.0, 29.25]
----------
3Reg (N=6, p=2): 83.75
10 out of 10
[7.15, 0.65, 5.8, 11.9, 30.2, 0.65, 11.05, 6.1, 2.55, 7.7]
----------
3Reg (N=6, p=3): 108.55
13 out of 13
[3.4, 6.0, 3.3, 5.75, 11.75, 23.4, 3.65, 3.65, 11.25, 7.5, 23.0, 3.3, 2.6]
----------
3Reg (N=6, p=4): 136.35
16 out of 16
[14.9, 7.7, 3.0, 2.55, 11.05, 11.9, 32.55, 19.45, 3.95, 3.95, 3.5, 5.8, 3.5, 5.75, 3.3, 3.5]
----------
3Reg (N=6, p=5): 179.0
19 out of 19
[3.25, 2.55, 19.05, 3.75, 3.75, 3.75, 5.5, 3.75, 11.95, 7.85, 3.75, 5.5, 25.6, 4.9, 5.85, 11.55, 6.1, 21.65, 28.95]
----------
3Reg (N=6, p=6): 201.6
22 out of 22
[34.6, 5.4, 5.75, 7.85, 2.6, 11.75, 3.65, 4.0, 14.4, 11.25, 5.75, 22.85, 3.65, 3.4, 2.25, 20.75, 4.0, 18.65, 3.65, 4.0, 5.4, 6.0]
----------
3Reg (N=6, p=7): 214.75
25 out of 25
[9.6, 3.85, 29.7, 3.45, 26.05, 19.95, 4.7, 5.5, 3.85, 7.55, 3.85, 4.25, 5.5, 4.25, 19.95, 3.45, 5.9, 12.05, 2.6, 3.85, 3.35, 3.45, 5.5, 11.3, 11.3]
-------

In [5]:
print("3-Regular graph (N=6, p=1,..,8):")
print(all_times)

3-Regular graph (N=6, p=1,..,8):
[72.0, 83.75, 108.55, 136.35, 179.0, 201.6, 214.75, 278.05]


### N=8

In [6]:
import os, re
times = [[] for _ in range(8)]

for filename in os.listdir('QAOA/3Reg/'):
    
    # job 59101659 was run for 3 regular graph with N=8
    if '59101659' in filename and filename.endswith('.out'):
        #print(filename)
        lines = open('QAOA/3Reg/%s' % filename).readlines()
        if len(lines) > 2 and lines[-2].startswith('TIME FOR SLICE IS:'):
            #print(re.split(';|=| |\*|\n', lines[1]))
            pi = int(re.split(';|=| |\*|\n', lines[1])[-4])
            #print(pi)
            time = float(lines[-1])
            #print(time)
            if time <= 100.0:
                times[pi-1].append(time)
n_s = [9,20,29,38,47,56,65,74]
all_times = []
for (i, t) in enumerate(times):
    print('----------')
    print("3Reg (N=8, p=%d): %s" % (i+1, np.round(sum(times[i]),4)))
    print("%d out of %d" % (len(times[i]), n_s[i]))
    print(times[i])
    all_times.append(np.round(sum(times[i]),4))

----------
3Reg (N=8, p=1): 111.95
9 out of 9
[11.45, 7.7, 7.25, 7.7, 4.4, 7.25, 11.9, 43.3, 11.0]
----------
3Reg (N=8, p=2): 184.5
20 out of 20
[11.0, 0.2, 7.25, 0.2, 11.9, 11.45, 11.45, 11.0, 7.25, 11.45, 11.45, 7.25, 11.9, 7.25, 41.45, 0.65, 0.2, 5.8, 7.7, 7.7]
----------
3Reg (N=8, p=3): 289.4
29 out of 29
[7.5, 7.85, 7.85, 11.05, 39.85, 11.4, 0.15, 3.3, 11.05, 11.75, 12.1, 0.85, 25.1, 11.75, 23.7, 11.75, 0.15, 7.85, 0.15, 7.15, 5.75, 11.75, 12.8, 7.5, 9.95, 7.15, 11.4, 7.15, 3.65]
----------
3Reg (N=8, p=4): 384.4
38 out of 38
[7.25, 11.45, 7.25, 3.5, 7.7, 7.7, 7.25, 3.5, 41.9, 11.0, 9.1, 11.9, 23.6, 9.55, 11.9, 0.65, 22.25, 11.9, 5.8, 7.25, 20.8, 11.9, 3.95, 8.15, 0.2, 11.45, 21.3, 7.7, 11.9, 7.25, 0.2, 11.9, 0.2, 7.25, 11.0, 3.95, 11.9, 11.0]
----------
3Reg (N=8, p=5): 498.9
47 out of 47
[7.25, 11.4, 7.85, 9.6, 11.4, 11.95, 27.8, 0.25, 26.6, 3.75, 34.85, 11.4, 11.95, 18.45, 20.75, 11.4, 11.4, 9.05, 0.25, 0.85, 7.25, 0.25, 3.75, 4.9, 3.75, 7.25, 7.85, 11.95, 7.25, 3.15, 26.0, 1

In [7]:
print("3-Regular graph (N=8, p=1,..,8):")
print(all_times)

3-Regular graph (N=8, p=1,..,8):
[111.95, 184.5, 289.4, 384.4, 498.9, 627.5, 757.4, 881.75]


## 3Reg Gate-Based

In [43]:
print('Times reflect unoptimized and unmapped -> after optimization (time goes down) --> after mapping (time goes up)')
unitaries = []
gate_times = []
for benchmark, N, layout, p in [('3Reg', 4, (2, 2), 1),
                              ('3Reg', 4, (2, 2), 2),
                              ('3Reg', 4, (2, 2), 3),
                              ('3Reg', 4, (2, 2), 4),
                              ('3Reg', 4, (2, 2), 5),
                              ('3Reg', 4, (2, 2), 6),
                              ('3Reg', 4, (2, 2), 7),
                              ('3Reg', 4, (2, 2), 8),
                              ('3Reg', 6, (2, 3), 1),
                              ('3Reg', 6, (2, 3), 2),
                              ('3Reg', 6, (2, 3), 3),
                              ('3Reg', 6, (2, 3), 4),
                              ('3Reg', 6, (2, 3), 5),
                              ('3Reg', 6, (2, 3), 6),
                              ('3Reg', 6, (2, 3), 7),
                              ('3Reg', 6, (2, 3), 8),
                              ('3Reg', 8, (2, 4), 1),
                              ('3Reg', 8, (2, 4), 2),
                              ('3Reg', 8, (2, 4), 3),
                              ('3Reg', 8, (2, 4), 4),
                              ('3Reg', 8, (2, 4), 5),
                              ('3Reg', 8, (2, 4), 6),
                              ('3Reg', 8, (2, 4), 7),
                              ('3Reg', 8, (2, 4), 8)]:
    unoptimized_unmapped_time = 0.0
    unmapped_time = 0.0
    mapped_time = 0.0
    samples = 1
    for _ in range(samples):
        random.seed(0)
        np.random.seed(1)
        tf.set_random_seed(2)
        circuit = qaoa.get_qaoa_circuit(N, p, benchmark)
        unoptimized_unmapped_time += np.round(util.circuitutil.get_max_pulse_time(circuit), 4)
        circuit = util.circuitutil.optimize_circuit(circuit)
        unmapped_time += np.round(util.circuitutil.get_max_pulse_time(circuit), 4)
        coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(layout[0], layout[1])
        circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)
        mapped_time += np.round(util.circuitutil.get_max_pulse_time(circuit), 4)
    template = 'benchmark: %s (N=%s, p=%s). Times %s --> optimized = %s --> mapped = %s'
    print(template % (benchmark, N, p, np.round(unoptimized_unmapped_time/samples,4), np.round(unmapped_time/samples,4), np.round(mapped_time/samples,4)))
    gate_times.append(np.round(mapped_time/samples,4))

Times reflect unoptimized and unmapped -> after optimization (time goes down) --> after mapping (time goes up)
benchmark: 3Reg (N=4, p=1). Times 51.9 --> optimized = 51.9 --> mapped = 81.5
benchmark: 3Reg (N=4, p=2). Times 99.1 --> optimized = 99.1 --> mapped = 158.3
benchmark: 3Reg (N=4, p=3). Times 146.3 --> optimized = 146.3 --> mapped = 235.1
benchmark: 3Reg (N=4, p=4). Times 193.5 --> optimized = 193.5 --> mapped = 311.9
benchmark: 3Reg (N=4, p=5). Times 240.7 --> optimized = 240.7 --> mapped = 388.7
benchmark: 3Reg (N=4, p=6). Times 287.9 --> optimized = 287.9 --> mapped = 465.5
benchmark: 3Reg (N=4, p=7). Times 335.1 --> optimized = 335.1 --> mapped = 542.3
benchmark: 3Reg (N=4, p=8). Times 382.3 --> optimized = 382.3 --> mapped = 619.1
benchmark: 3Reg (N=6, p=1). Times 51.9 --> optimized = 51.9 --> mapped = 113.2
benchmark: 3Reg (N=6, p=2). Times 102.5 --> optimized = 102.5 --> mapped = 199.3
benchmark: 3Reg (N=6, p=3). Times 153.1 --> optimized = 153.1 --> mapped = 277.4
bench

In [44]:
print("=== Gate Based ===")
print("3-Regular graph (N=4, p=1,..,8):")
print(gate_times[:8])
print("3-Regular graph (N=6, p=1,..,8):")
print(gate_times[8:16])
print("3-Regular graph (N=8, p=1,..,8):")
print(gate_times[16:])

=== Gate Based ===
3-Regular graph (N=4, p=1,..,8):
[81.5, 158.3, 235.1, 311.9, 388.7, 465.5, 542.3, 619.1]
3-Regular graph (N=6, p=1,..,8):
[113.2, 199.3, 277.4, 355.5, 433.6, 511.7, 589.8, 667.9]
3-Regular graph (N=8, p=1,..,8):
[162.5, 364.6, 529.8, 695.0, 860.2, 1025.4, 1190.6, 1355.8]


## 3Reg Strict Compilation

### N=6

In [19]:
import os, re
times = [[] for _ in range(8)]

for filename in os.listdir('QAOA/3Reg/'):
    
    # job 59157192 was run for 3 regular graph with N=6
    if '59157192' in filename and filename.endswith('.out'):
        #print(filename)
        lines = open('QAOA/3Reg/%s' % filename).readlines()
        if len(lines) > 2 and lines[-2].startswith('TIME FOR SLICE IS:'):
            #print(re.split(';|=| |\*|\n', lines[1]))
            pi = int(re.split(';|=| |\*|\n', lines[1])[-4])
            #print(pi)
            time = float(lines[-1])
            #print(time)
            if time <= 100.0:
                times[pi-1].append(time)
n_s = [7,10,13,16,19,22,25,28]
all_times = []
for (i, t) in enumerate(times):
    print('---------- Strict ----------')
    print("3Reg (N=6, p=%d): %s" % (i+1, np.round(sum(times[i]),4)))
    print("%d out of %d" % (len(times[i]), n_s[i]))
    print(times[i])
    all_times.append(np.round(sum(times[i]),4))

---------- Strict ----------
3Reg (N=6, p=1): 0
0 out of 7
[]
---------- Strict ----------
3Reg (N=6, p=2): 0
0 out of 10
[]
---------- Strict ----------
3Reg (N=6, p=3): 0
0 out of 13
[]
---------- Strict ----------
3Reg (N=6, p=4): 0
0 out of 16
[]
---------- Strict ----------
3Reg (N=6, p=5): 0
0 out of 19
[]
---------- Strict ----------
3Reg (N=6, p=6): 0
0 out of 22
[]
---------- Strict ----------
3Reg (N=6, p=7): 0
0 out of 25
[]
---------- Strict ----------
3Reg (N=6, p=8): 0
0 out of 28
[]


### N=8

In [22]:
import os, re
times = [[] for _ in range(8)]

for filename in os.listdir('QAOA/3Reg/'):
    
    # job 59157313 was run for 3 regular graph with N=6
    if '59157313' in filename and filename.endswith('.out'):
        #print(filename)
        lines = open('QAOA/3Reg/%s' % filename).readlines()
        if len(lines) > 2 and lines[-2].startswith('TIME FOR SLICE IS:'):
            #print(re.split(';|=| |\*|\n', lines[1]))
            pi = int(re.split(';|=| |\*|\n', lines[1])[-4])
            #print(pi)
            time = float(lines[-1])
            #print(time)
            if time <= 100.0:
                times[pi-1].append(time)
n_s = [9,20,29,38,47,56,65,74]
all_times = []
for (i, t) in enumerate(times):
    print('---------- Strict ----------')
    print("3Reg (N=8, p=%d): %s" % (i+1, np.round(sum(times[i]),4)))
    print("%d out of %d" % (len(times[i]), n_s[i]))
    print(times[i])
    all_times.append(np.round(sum(times[i]),4))

---------- Strict ----------
3Reg (N=8, p=1): 0
0 out of 9
[]
---------- Strict ----------
3Reg (N=8, p=2): 0
0 out of 20
[]
---------- Strict ----------
3Reg (N=8, p=3): 0
0 out of 29
[]
---------- Strict ----------
3Reg (N=8, p=4): 0
0 out of 38
[]
---------- Strict ----------
3Reg (N=8, p=5): 0
0 out of 47
[]
---------- Strict ----------
3Reg (N=8, p=6): 0
0 out of 56
[]
---------- Strict ----------
3Reg (N=8, p=7): 0
0 out of 65
[]
---------- Strict ----------
3Reg (N=8, p=8): 0
0 out of 74
[]


---

## Erdos-Renyi Full QOC

### N=6

In [8]:
import os, re
times = [[] for _ in range(8)]

for filename in os.listdir('QAOA/ErdosRenyi/'):
    
    # job 59111032-[0-88] and 59111920-[89-121] were run for ErdosRenyi graph with N=6
    if '59111032' in filename and filename.endswith('.out'):
        #print(filename)
        lines = open('QAOA/ErdosRenyi/%s' % filename).readlines()
        if len(lines) > 2 and lines[-2].startswith('TIME FOR SLICE IS:'):
            #print(re.split(';|=| |\*|\n', lines[1]))
            pi = int(re.split(';|=| |\*|\n', lines[1])[-4])
            #print(pi)
            time = float(lines[-1])
            #print(time)
            if time <= 100.0:
                times[pi-1].append(time)
    if '59111920' in filename and filename.endswith('.out'):
        #print(filename)
        lines = open('QAOA/ErdosRenyi/%s' % filename).readlines()
        if len(lines) > 2 and lines[-2].startswith('TIME FOR SLICE IS:'):
            #print(re.split(';|=| |\*|\n', lines[1]))
            pi = int(re.split(';|=| |\*|\n', lines[1])[-4])
            #print(pi)
            time = float(lines[-1])
            #print(time)
            if time <= 100.0:
                times[pi-1].append(time)

n_s = [3,8,11,14,17,20,23,26]
all_times = []
for (i, t) in enumerate(times):
    print('----------')
    print("ErdosRenyi (N=6, p=%d): %s" % (i+1, np.round(sum(times[i]),4)))
    print("%d out of %d" % (len(times[i]), n_s[i]))
    print(times[i])
    all_times.append(np.round(sum(times[i]),4))

----------
ErdosRenyi (N=6, p=1): 26.55
3 out of 3
[7.7, 7.85, 11.0]
----------
ErdosRenyi (N=6, p=2): 42.7
8 out of 8
[0.85, 7.85, 7.7, 0.65, 11.0, 7.25, 0.15, 7.25]
----------
ErdosRenyi (N=6, p=3): 71.95
11 out of 11
[11.4, 0.85, 7.6, 7.85, 7.05, 7.5, 11.05, 0.25, 3.4, 7.5, 7.5]
----------
ErdosRenyi (N=6, p=4): 104.1
14 out of 14
[0.15, 7.25, 0.65, 7.25, 2.95, 8.2, 7.85, 7.25, 7.15, 10.5, 11.45, 18.5, 7.7, 7.25]
----------
ErdosRenyi (N=6, p=5): 141.25
17 out of 17
[7.2, 7.25, 0.2, 19.0, 7.85, 8.05, 10.8, 7.25, 19.6, 7.6, 9.6, 8.05, 7.85, 7.25, 0.85, 7.85, 5.0]
----------
ErdosRenyi (N=6, p=6): 196.85
20 out of 20
[25.45, 7.15, 4.95, 7.6, 19.1, 7.5, 0.25, 7.5, 7.5, 8.1, 7.5, 11.05, 7.6, 7.15, 22.65, 7.05, 0.85, 7.15, 22.65, 8.1]
----------
ErdosRenyi (N=6, p=7): 216.1
23 out of 23
[9.55, 7.65, 7.55, 18.6, 7.95, 28.05, 7.55, 26.45, 0.3, 8.25, 7.65, 0.6, 3.95, 8.25, 10.0, 7.55, 7.55, 7.55, 7.55, 7.65, 7.15, 7.55, 11.2]
----------
ErdosRenyi (N=6, p=8): 242.3
26 out of 26
[18.0, 8.2, 

In [9]:
print("ErdosRenyi graph (N=6, p=1,..,8):")
print(all_times)

ErdosRenyi graph (N=6, p=1,..,8):
[26.55, 42.7, 71.95, 104.1, 141.25, 196.85, 216.1, 242.3]


### N=8

In [16]:
import os, re
times = [[] for _ in range(8)]

for filename in os.listdir('QAOA/ErdosRenyi/'):
    
    # job 59118765 was run for ErdosRenyi graph with N=8
    if '59118765' in filename and filename.endswith('.out'):
        #print(filename)
        lines = open('QAOA/ErdosRenyi/%s' % filename).readlines()
        if len(lines) > 2 and lines[-2].startswith('TIME FOR SLICE IS:'):
            #print(re.split(';|=| |\*|\n', lines[1]))
            pi = int(re.split(';|=| |\*|\n', lines[1])[-4])
            #print(pi)
            time = float(lines[-1])
            #print(time)
            if time <= 200.0:
                times[pi-1].append(time)
        else:
            print("%s did not finish." % filename)

n_s = [7,13,19,25,31,37,43,49]
all_times = []
for (i, t) in enumerate(times):
    print('----------')
    print("ErdosRenyi (N=8, p=%d): %s" % (i+1, np.round(sum(times[i]),4)))
    print("%d out of %d" % (len(times[i]), n_s[i]))
    print(times[i])
    all_times.append(np.round(sum(times[i]),4))

arrayJob_59118765_206.out did not finish.
arrayJob_59118765_151.out did not finish.
arrayJob_59118765_157.out did not finish.
arrayJob_59118765_163.out did not finish.
arrayJob_59118765_145.out did not finish.
arrayJob_59118765_120.out did not finish.
arrayJob_59118765_212.out did not finish.
arrayJob_59118765_200.out did not finish.
arrayJob_59118765_194.out did not finish.
----------
ErdosRenyi (N=8, p=1): 81.6
7 out of 7
[2.55, 11.45, 7.25, 33.95, 7.7, 7.25, 11.45]
----------
ErdosRenyi (N=8, p=2): 143.75
13 out of 13
[7.25, 11.0, 11.0, 11.9, 0.65, 7.25, 16.1, 14.25, 7.25, 13.8, 11.0, 21.3, 11.0]
----------
ErdosRenyi (N=8, p=3): 235.25
19 out of 19
[11.75, 7.5, 47.25, 11.75, 7.5, 11.75, 11.05, 7.85, 11.05, 7.5, 9.95, 9.25, 11.05, 15.25, 11.4, 10.65, 7.5, 13.5, 11.75]
----------
ErdosRenyi (N=8, p=4): 329.0
25 out of 25
[9.1, 7.25, 11.45, 11.0, 59.75, 30.2, 11.45, 7.25, 13.3, 11.9, 10.5, 18.95, 10.05, 11.45, 7.25, 11.45, 8.15, 7.25, 10.05, 14.75, 7.25, 11.0, 11.45, 7.25, 9.55]
-----

In [14]:
print("ErdosRenyi graph (N=8, p=1,..,8):")
print(all_times)

ErdosRenyi graph (N=8, p=1,..,8):
[81.6, 143.75, 235.25, 329.0, 513.65, 487.1, 441.6, 562.8]


## Erdos-Renyi Full QOC

### N=6

## Erdos-Renyi Gate-Based

In [45]:
print('Times reflect unoptimized and unmapped -> after optimization (time goes down) --> after mapping (time goes up)')
unitaries = []
erdosrenyi_gate_times = []
for benchmark, N, layout, p in [('ErdosRenyi', 4, (2, 2), 1),
                              ('ErdosRenyi', 4, (2, 2), 2),
                              ('ErdosRenyi', 4, (2, 2), 3),
                              ('ErdosRenyi', 4, (2, 2), 4),
                              ('ErdosRenyi', 4, (2, 2), 5),
                              ('ErdosRenyi', 4, (2, 2), 6),
                              ('ErdosRenyi', 4, (2, 2), 7),
                              ('ErdosRenyi', 4, (2, 2), 8),
                              ('ErdosRenyi', 6, (2, 3), 1),
                              ('ErdosRenyi', 6, (2, 3), 2),
                              ('ErdosRenyi', 6, (2, 3), 3),
                              ('ErdosRenyi', 6, (2, 3), 4),
                              ('ErdosRenyi', 6, (2, 3), 5),
                              ('ErdosRenyi', 6, (2, 3), 6),
                              ('ErdosRenyi', 6, (2, 3), 7),
                              ('ErdosRenyi', 6, (2, 3), 8),
                              ('ErdosRenyi', 8, (2, 4), 1),
                              ('ErdosRenyi', 8, (2, 4), 2),
                              ('ErdosRenyi', 8, (2, 4), 3),
                              ('ErdosRenyi', 8, (2, 4), 4),
                              ('ErdosRenyi', 8, (2, 4), 5),
                              ('ErdosRenyi', 8, (2, 4), 6),
                              ('ErdosRenyi', 8, (2, 4), 7),
                              ('ErdosRenyi', 8, (2, 4), 8)]:
    unoptimized_unmapped_time = 0.0
    unmapped_time = 0.0
    mapped_time = 0.0
    samples = 1
    for _ in range(samples):
        random.seed(0)
        np.random.seed(1)
        tf.set_random_seed(2)
        circuit = qaoa.get_qaoa_circuit(N, p, benchmark)
        unoptimized_unmapped_time += np.round(util.circuitutil.get_max_pulse_time(circuit), 4)
        circuit = util.circuitutil.optimize_circuit(circuit)
        unmapped_time += np.round(util.circuitutil.get_max_pulse_time(circuit), 4)
        coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(layout[0], layout[1])
        circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)
        mapped_time += np.round(util.circuitutil.get_max_pulse_time(circuit), 4)
    template = 'benchmark: %s (N=%s, p=%s). Times %s --> optimized = %s --> mapped = %s'
    print(template % (benchmark, N, p, np.round(unoptimized_unmapped_time/samples,4), np.round(unmapped_time/samples,4), np.round(mapped_time/samples,4)))
    erdosrenyi_gate_times.append(np.round(mapped_time/samples,4))

Times reflect unoptimized and unmapped -> after optimization (time goes down) --> after mapping (time goes up)
benchmark: ErdosRenyi (N=4, p=1). Times 19.9 --> optimized = 19.9 --> mapped = 34.5
benchmark: ErdosRenyi (N=4, p=2). Times 38.4 --> optimized = 38.4 --> mapped = 53.0
benchmark: ErdosRenyi (N=4, p=3). Times 56.9 --> optimized = 56.9 --> mapped = 71.5
benchmark: ErdosRenyi (N=4, p=4). Times 75.4 --> optimized = 75.4 --> mapped = 90.0
benchmark: ErdosRenyi (N=4, p=5). Times 93.9 --> optimized = 93.9 --> mapped = 108.5
benchmark: ErdosRenyi (N=4, p=6). Times 112.4 --> optimized = 112.4 --> mapped = 127.0
benchmark: ErdosRenyi (N=4, p=7). Times 130.9 --> optimized = 130.9 --> mapped = 145.5
benchmark: ErdosRenyi (N=4, p=8). Times 149.4 --> optimized = 149.4 --> mapped = 164.0
benchmark: ErdosRenyi (N=6, p=1). Times 35.9 --> optimized = 35.9 --> mapped = 83.7
benchmark: ErdosRenyi (N=6, p=2). Times 70.4 --> optimized = 70.4 --> mapped = 151.2
benchmark: ErdosRenyi (N=6, p=3). Time

In [47]:
print("=== Gate Based ===")
print("Erdos-Renyi graph (N=4, p=1,..,8):")
print(erdosrenyi_gate_times[:8])
print("Erdos-Renyi graph (N=6, p=1,..,8):")
print(erdosrenyi_gate_times[8:16])
print("Erdos-Renyi graph (N=8, p=1,..,8):")
print(erdosrenyi_gate_times[16:])

=== Gate Based ===
Erdos-Renyi graph (N=4, p=1,..,8):
[34.5, 53.0, 71.5, 90.0, 108.5, 127.0, 145.5, 164.0]
Erdos-Renyi graph (N=6, p=1,..,8):
[83.7, 151.2, 223.4, 295.6, 367.8, 440.0, 512.2, 584.4]
Erdos-Renyi graph (N=8, p=1,..,8):
[157.1, 297.0, 442.9, 596.2, 749.5, 902.8, 1056.1, 1209.4]
